In [3]:
from munch import Munch
from tf.app import use

In [4]:
B = use("etcbc/dss", hoist=globals())
DSS = Munch({"F": F, "L": L, "T": T, "name": "DSS"})

A = use("etcbc/bhsa", hoist=globals())
BHSA = Munch({"F": F, "L": L, "T": T, "name": "BHSA"})

This is Text-Fabric 10.1.0
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

71 features found and 1 ignored


This is Text-Fabric 10.1.0
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

122 features found and 0 ignored


In [63]:
del F, L, T

In [5]:
%load_ext autoreload
%autoreload 2

In [46]:
import os
import collections
from itertools import chain
from collections import defaultdict

In [48]:
dss_sections = {}
for word in DSS.F.otype.s("word"):    
    scroll = DSS.T.scrollName(DSS.L.u(word, "scroll")[0])
    book = DSS.F.book_etcbc.v(word)    
    chapter = DSS.F.chapter.v(word)
    verse = DSS.F.verse.v(word)    
    if None in (scroll, book, chapter, verse):
        continue
    section = (book, chapter, verse)
    dss_sections.setdefault(section, {}).setdefault(scroll, []).append(word)

In [146]:
def remove_duplicates(iterable):
    return list(dict.fromkeys(iterable))


class TFOb:
    def __init__(self, ids, source):
        if type(ids) is int:
            ids = [ids]
        self.ids = remove_duplicates(ids)
        self.source = source
        
    @classmethod
    def section(self, section, source, scroll=None):
        if source.name == "BHSA":
            return TFOb(source.T.nodeFromSection(section), source)
        
        section = (section[0], str(section[1]), str(section[2]))
        dss_section = dss_sections[section]
        if scroll is None:
            scroll = list(dss_section.keys())[0]
        return TFOb(dss_section[scroll], source)
        
    def __getattr__(self, attr):
        if self.source.name == "DSS" and attr == "lex":
            attr = "lex_etcbc"
            
        feature = getattr(self.source.F, attr)
        return [getattr(self.source.F, attr).v(id_) for id_ in self.ids]

    def up(self, otype=None):
        if otype == self.kind:
            return TFOb(self.ids.copy())
        return TFOb(
            chain(*[self.source.L.u(id_, otype) for id_ in self.ids]),
            source=self.source,
        )

    def down(self, otype=None):
        if otype == self.kind:
            return TFOb(self.ids.copy(), self.source)
        return TFOb(
            chain(*[self.source.L.d(id_, otype) for id_ in self.ids]),
            source=self.source,
        )

    def filter(self, **kwargs):
        ids = []
        for id_ in self.ids:
            for key, value in kwargs.items():
                if key == "lex" and self.source.name == "DSS":
                    key = "lex_etcbc"
                if getattr(self.source.F, key).v(id_) != value:
                    break
            else:
                ids.append(id_)
        return TFOb(ids, source=self.source)

    def first(self, **kwargs):
        ids = []
        for id_ in self.ids:
            for key, value in kwargs.items():
                if key == "lex" and self.source.name == "DSS":
                    key = "lex_etcbc"
                if getattr(self.source.F, key).v(id_) != value:
                    break
            else:
                return TFOb(id_, source=self.source)
        return TFOb([], source=self.source)
            
    def __getitem__(self, i):
        return TFOb(self.ids[i], source=self.source)

    @property
    def text(self):
        return self.source.T.text(self.ids)
    
    def __str__(self):
        if self.kind not in ("word", "NoNode"):
            return str(self.down("word"))
        return " ".join([g_cons for g_cons in self.g_cons if g_cons])

    @property
    def kind(self):
        if len(self.ids) > 0:
            return self.otype[0]
        return "NoNode"

    def __repr__(self):
        return f'<{self.kind} "{str(self)}">'

In [147]:
TFOb.section(["Isaiah", 20, 6], BHSA)

<verse "W >MR JCB H >J H ZH B JWM H HW> HNH KH MBVNW >CR NSNW CM L <ZRH L HNYL M PNJ MLK >CWR W >JK NMLV >NXNW">

In [148]:
TFOb.section(["Genesis", 1, 1], BHSA).down("word").first(lex="BR>[").up("clause").down("phrase").first(function="Subj")

<phrase ">LHJM">

In [149]:
verb = TFOb.section(["Genesis", 1, 1], BHSA).down("word").first(lex="BR>[")
verb.up("clause").down("phrase").first(function="Subj")

<phrase ">LHJM">

In [150]:
verb = TFOb.section(["Genesis", 1, 1], DSS).down("word").first(lex="BR>[")
verb.up("clause").down("phrase").first(function="Subj")

<NoNode "">

In [125]:
TFOb.section(["Genesis", 1, 1], BHSA).down("word").first(lex="BR>[")

<word "BR>">

In [120]:
TFOb.section(["Genesis", 1, 1], BHSA).down("word").lex

['B', 'R>CJT/', 'BR>[', '>LHJM/', '>T', 'H', 'CMJM/', 'W', '>T', 'H', '>RY/']

In [79]:
TFOb.section(["Isaiah", 20, 6], DSS)

<word ואמר יושב האיי הזה ביום ההוא הנה כה מבט׳נו אשר נסמך שמ לעזרה להנצל מפני מלך אשור ואיך נמלט אנחנו ׃ >

In [34]:
TFOb.section(["Isaiah", 6, 6], BHSA)

<verse וַיָּ֣עָף אֵלַ֗י אֶחָד֙ מִן־הַשְּׂרָפִ֔ים וּבְיָדֹ֖ו רִצְפָּ֑ה בְּמֶ֨לְקַחַ֔יִם לָקַ֖ח מֵעַ֥ל הַמִּזְבֵּֽחַ׃ >

In [37]:
TFOb.section(["Isaiah", 6, 6], BHSA).down("word").lex

['W',
 '<WP[',
 '>L',
 '>XD/',
 'MN',
 'H',
 'FRP/',
 'W',
 'B',
 'JD/',
 'RYPH/',
 'B',
 'MLQXJM/',
 'LQX[',
 'MN',
 '<L',
 'H',
 'MZBX/']

In [17]:
word = TFOb(1033, DSS)
word

<sign אָדָ֔ם >

In [20]:
TFOb(1, BHSA).up("clause").down("phrase").first(function="Subj")

<phrase אֱלֹהִ֑ים >

In [221]:
word.up("clause").down("phrase")

<phrase וַיָּבֵא֙ אֶל־הָ֣אָדָ֔ם >

In [223]:
word.up("clause").down("phrase")[0]

TypeError: 'TFOb' object is not subscriptable

In [191]:
clause.down("phrase").filter(function="toto")

<phrase וַיָּבֵא֙ אֶל־הָ֣אָדָ֔ם >

In [167]:
phrases.function

['Conj', 'Pred', 'Cmpl']

In [6]:
def verse_lex(section):
    verse = (T.nodeFromSection(section))
    return [F.lex.v(w) for w in L.d(verse, "word")]

In [7]:
verse_lex(("Genesis", 12, 6))

['W',
 '<BR[',
 '>BRM/',
 'B',
 'H',
 '>RY/',
 '<D',
 'MQWM/',
 'CKM=/',
 '<D',
 '>LWN/',
 'MWRH===/',
 'W',
 'H',
 'KN<NJ/',
 '>Z',
 'B',
 'H',
 '>RY/']

In [6]:
A.indent(reset=True)
A.info("Counting nodes ...")

i = 0
for n in N.walk():
    i += 1

A.info("{} nodes".format(i))

  0.00s Counting nodes ...
  0.20s 1446831 nodes


In [8]:
F.otype.slotType

'word'

In [8]:
A.indent(reset=True)
A.info("counting objects ...")

for otype in F.otype.all:
    i = 0

    A.indent(level=1, reset=True)

    for n in F.otype.s(otype):
        i += 1

    A.info("{:>7} {}s".format(i, otype))

A.indent(level=0)
A.info("Done")

  0.00s counting objects ...
   |     0.00s      39 books
   |     0.00s     929 chapters
   |     0.00s    9230 lexs
   |     0.00s   23213 verses
   |     0.01s   45179 half_verses
   |     0.01s   63717 sentences
   |     0.01s   64514 sentence_atoms
   |     0.01s   88131 clauses
   |     0.01s   90704 clause_atoms
   |     0.02s  253203 phrases
   |     0.02s  267532 phrase_atoms
   |     0.01s  113850 subphrases
   |     0.04s  426590 words
  0.15s Done


In [9]:
verbs = collections.Counter()
A.indent(reset=True)
A.info("Collecting data")

lines = []
n = 0
for w in N.walk():
    # for w in F.otype.s("lex"):
    if F.lex.v(w) == "BW>[" and F.sp.v(w) == "verb" and F.otype.v(w) == "word":
        n += 1
        if n <= 3:
            continue
        section = T.sectionFromNode(w)
        lines.append(
            {
                "node": w,
                "tense": F.vt.v(w),
                "stem": F.st.v(w),
                "book": section[0],
                "chapter": section[1],
                "verse": section[2],
            }
        )
        verse = L.u(w, "verse")
        print("verse", verse, T.text(verse))
        
        clause = L.u(w, "clause")[0]
        print("clause", clause, T.text(clause))
        sentence = L.u(w, "sentence")
        print("sentence", sentence, T.text(sentence))

        clause_phrases = L.d(clause, "phrase")
        print("clause_phrases", clause_phrases)
        
        subject = None
        for phrase in clause_phrases:
            if F.function.v(phrase) == "Subj":
                subject = phrase
                break
                
        print("subject", subject, T.text(subject))
        
        cmpls = [phrase for phrase in clause_phrases if F.function.v(phrase) == "Cmpl"]
        
        print("Complement", cmpls, [T.text(cmpl) for cmpl in cmpls])
 
            
        for word in clause_words:
            print(F.sp.v(word))
        # L.d(clause, sp="verb")
        # print("subject", subject, T.text(subject))
        break
        # print(T.sectionFromNode(w))
        # print("book", F.book.v(w))
        # print(F.lex_utf8.v(w))
        # print(F.vt.v(w))
        # print(F.vs.v(w))

A.info("Done")

  0.00s Collecting data
verse (1414472,) וְהֶ֨בֶל הֵבִ֥יא גַם־ה֛וּא מִבְּכֹרֹ֥ות צֹאנֹ֖ו וּמֵֽחֶלְבֵהֶ֑ן וַיִּ֣שַׁע יְהוָ֔ה אֶל־הֶ֖בֶל וְאֶל־מִנְחָתֹֽו׃ 
clause 427895 וְהֶ֨בֶל הֵבִ֥יא גַם־ה֛וּא מִבְּכֹרֹ֥ות צֹאנֹ֖ו וּמֵֽחֶלְבֵהֶ֑ן 
sentence (1172549,) וְהֶ֨בֶל הֵבִ֥יא גַם־ה֛וּא מִבְּכֹרֹ֥ות צֹאנֹ֖ו וּמֵֽחֶלְבֵהֶ֑ן 
clause_phrases (652573, 652574, 652575, 652576)
subject 652574 הֶ֨בֶל גַם־ה֛וּא 
Complement [652576] ['מִבְּכֹרֹ֥ות צֹאנֹ֖ו וּמֵֽחֶלְבֵהֶ֑ן ']


NameError: name 'clause_words' is not defined

In [12]:
F.lex

In [62]:
import pandas as pd

In [63]:
df = pd.DataFrame(lines)
df

,node,tense,stem,book,chapter,verse
0,1030,wayq,NA,Genesis,2,19
1,1112,wayq,NA,Genesis,2,22
2,1710,wayq,NA,Genesis,4,3
3,1721,perf,NA,Genesis,4,4
4,2676,impf,NA,Genesis,6,4
...,...,...,...,...,...,...
2565,425959,wayq,NA,2_Chronicles,35,22
2566,426143,wayq,NA,2_Chronicles,36,4
2567,426186,perf,NA,2_Chronicles,36,7
2568,426253,wayq,NA,2_Chronicles,36,10


In [64]:
df.to_csv("BO.csv", index=False)

In [42]:
Fall()

['book',
 'book@am',
 'book@ar',
 'book@bn',
 'book@da',
 'book@de',
 'book@el',
 'book@en',
 'book@es',
 'book@fa',
 'book@fr',
 'book@he',
 'book@hi',
 'book@id',
 'book@ja',
 'book@ko',
 'book@la',
 'book@nl',
 'book@pa',
 'book@pt',
 'book@ru',
 'book@sw',
 'book@syc',
 'book@tr',
 'book@ur',
 'book@yo',
 'book@zh',
 'chapter',
 'code',
 'det',
 'domain',
 'freq_lex',
 'function',
 'g_cons',
 'g_cons_utf8',
 'g_lex',
 'g_lex_utf8',
 'g_word',
 'g_word_utf8',
 'gloss',
 'gn',
 'label',
 'language',
 'lex',
 'lex_utf8',
 'ls',
 'nametype',
 'nme',
 'nu',
 'number',
 'otype',
 'pargr',
 'pdp',
 'pfm',
 'phono',
 'phono_trailer',
 'prs',
 'prs_gn',
 'prs_nu',
 'prs_ps',
 'ps',
 'qere',
 'qere_trailer',
 'qere_trailer_utf8',
 'qere_utf8',
 'rank_lex',
 'rela',
 'sp',
 'st',
 'tab',
 'trailer',
 'trailer_utf8',
 'txt',
 'typ',
 'uvf',
 'vbe',
 'vbs',
 'verse',
 'voc_lex',
 'voc_lex_utf8',
 'vs',
 'vt']

In [9]:
for name in Fall():
    print(name, getattr(F, name).v(1033))

book None
book@am None
book@ar None
book@bn None
book@da None
book@de None
book@el None
book@en None
book@es None
book@fa None
book@fr None
book@he None
book@hi None
book@id None
book@ja None
book@ko None
book@la None
book@nl None
book@pa None
book@pt None
book@ru None
book@sw None
book@syc None
book@tr None
book@ur None
book@yo None
book@zh None
chapter None
code None
det None
domain None
freq_lex 553
function None
g_cons >DM
g_cons_utf8 אדם
g_lex >@D@M
g_lex_utf8 אָדָם
g_word >@D@80M
g_word_utf8 אָדָ֔ם
gloss human, mankind
gn m
label None
language Hebrew
lex >DM/
lex_utf8 אדם
ls none
nametype None
nme 
nu sg
number 1033
otype word
pargr None
pdp subs
pfm n/a
phono ʔāḏˈām
phono_trailer  
prs absent
prs_gn NA
prs_nu NA
prs_ps NA
ps NA
qere None
qere_trailer None
qere_trailer_utf8 None
qere_utf8 None
rank_lex 93
rela None
sp subs
st a
tab None
trailer  
trailer_utf8  
txt None
typ None
uvf absent
vbe n/a
vbs n/a
verse None
voc_lex >@D@M
voc_lex_utf8 אָדָם
vs NA
vt NA


In [56]:
F.st.v(935)

'a'

In [59]:
getattr(F, "st").v(935)

'a'

In [60]:
for name in Fall():
    print(name, getattr(F, name).v(1437760))
    print(name, getattr(F, name).v(935))

book None
book None
book@am None
book@am None
book@ar None
book@ar None
book@bn None
book@bn None
book@da None
book@da None
book@de None
book@de None
book@el None
book@el None
book@en None
book@en None
book@es None
book@es None
book@fa None
book@fa None
book@fr None
book@fr None
book@he None
book@he None
book@hi None
book@hi None
book@id None
book@id None
book@ja None
book@ja None
book@ko None
book@ko None
book@la None
book@la None
book@nl None
book@nl None
book@pa None
book@pa None
book@pt None
book@pt None
book@ru None
book@ru None
book@sw None
book@sw None
book@syc None
book@syc None
book@tr None
book@tr None
book@ur None
book@ur None
book@yo None
book@yo None
book@zh None
book@zh None
chapter None
chapter None
code None
code None
det None
det None
domain None
domain None
freq_lex 1547
freq_lex 1547
function None
function None
g_cons None
g_cons HLK
g_cons_utf8 None
g_cons_utf8 הלך
g_lex None
g_lex HOL;K:
g_lex_utf8 None
g_lex_utf8 הֹלֵךְ
g_word None
g_word HOL;73K:
g_word_utf8 None